# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
print(data_0.shape)
display(data_0.info())
display(data_0.head(5))

(100000, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
print(data_1.shape)
display(data_1.info())
display(data_1.head(5))

(100000, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
print(data_2.shape)
display(data_2.info())
display(data_2.head(5))

(100000, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<div class="alert alert-warning">
<h2> Комментарий ревьюера ⚠️ <a class="tocSkip"> </h2>

В данном проекте мы работаем с данными одинаковой природы, а это значит, что большинство расчетов для регионов будут повторяться. Это отличная возможность потренироваться над оптимизацией кода. Например, вот здесь. Первичное знакомство с данными можно было реализовать в функции и вызывать эту функцию для каждого из датафреймов (в цикле в идеале). В нее можно включить все основные методы типа `head`, `info` и т.д.  Можно графики строить какие-нибудь... Из готового, я бы рекомендовал почитать про [pandas_profiling](https://habr.com/ru/company/ruvds/blog/451478/?)


In [6]:
data_0 = data_0.drop(['id'], axis=1).copy()
data_1 = data_1.drop(['id'], axis=1).copy()
data_2 = data_2.drop(['id'], axis=1).copy()

<div class="alert alert-warning">
<h2> Комментарий ревьюера ⚠️ <a class="tocSkip"> </h2>

Не всегда информация подобного рода бывает бесполезной, в нашем случае, например, по id можно посмотреть дубликаты пользователей, возможно был какой-то баг, о котором хорошо бы сообщить заказчику


**Вывод**

Были проанализированы 3 датасета. Пропуски в столбцах отсутствуют. Также все датасеты имеют одинаковый размер. На данном этапы были удалены дубликаты, если таковые имелись. Также датасеты не были объединены, так как интересует выбор наиболее прибыльного региона. Также был удален столбец с индексом скважины.

## Обучение и проверка модели

***Первый датасет***

In [7]:
target_0 = data_0['product']
features_0 = data_0.drop(['product'], axis=1)

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

<div class="alert alert-success">
<h2> Комментарий ревьюера ✔️ <a class="tocSkip"> </h2>

Круто, что ты фиксируешь `random_state`. Кстати, его можно задать один раз в начале проекта, например, используя любимый нами [numpy](https://pyprog.pro/random_sampling_functions/seed.html?)


In [8]:
model_reg_0 = LinearRegression()
model_reg_0.fit(features_train_0, target_train_0)

predictions_valid_reg_0 = model_reg_0.predict(features_valid_0)

result_reg_0 = (mean_squared_error(target_valid_0, predictions_valid_reg_0))**0.5

print('Средняя квадратичная ошибка на первом датасете по модели линейной регрессии:', result_reg_0)

Средняя квадратичная ошибка на первом датасете по модели линейной регрессии: 37.5794217150813


<div class="alert alert-success">
<h2> Комментарий ревьюера ✔️ <a class="tocSkip"> </h2>

У метрики `MSE` есть встроенная возможность сразу считать rmse. Вот [здесь](https://stackoverflow.com/questions/17197492/is-there-a-library-function-for-root-mean-square-error-rmse-in-python) про это пишут


***Второй датасет***

In [9]:
target_1 = data_1['product']
features_1 = data_1.drop(['product'], axis=1)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

In [10]:
model_reg_1 = LinearRegression()
model_reg_1.fit(features_train_1, target_train_1)

predictions_valid_reg_1 = model_reg_1.predict(features_valid_1)

result_reg_1 = (mean_squared_error(target_valid_1, predictions_valid_reg_1))**0.5

print('Средняя квадратичная ошибка на первом датасете по модели линейной регрессии:', result_reg_1)

Средняя квадратичная ошибка на первом датасете по модели линейной регрессии: 0.893099286775617


<div class="alert alert-success">
<h2> Комментарий ревьюера ✔️ <a class="tocSkip"> </h2>

Для линейных моделей важно, чтобы признаки имели схожий порядок. В нашем случае нам повезло, признаки не сильно отличаются дипазоном изменения значений. Но не стоит про это забывать. Вообще, полезно попробовать построить модели на разных данных, и с масштабированием и без, только так, пробуя разные варианты, можно найти лучший


***Третий датасет***

In [11]:
target_2 = data_2['product']
features_2 = data_2.drop(['product'], axis=1)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

**Линейная регрессия**

In [12]:
model_reg_2 = LinearRegression()
model_reg_2.fit(features_train_2, target_train_2)

predictions_valid_reg_2 = model_reg_2.predict(features_valid_2)

result_reg_2 = (mean_squared_error(target_valid_2, predictions_valid_reg_2))**0.5

print('Средняя квадратичная ошибка на первом датасете по модели линейной регрессии:', result_reg_2)

Средняя квадратичная ошибка на первом датасете по модели линейной регрессии: 40.02970873393434


In [13]:
print('Первый регион')
print(f'Среднее значение предсказанных запасов нефти в первом регионе в скважине составляет {predictions_valid_reg_0.mean()}')
print(f'RMSE модели составляет {result_reg_0}')

print('Второй регион')
print(f'Среднее значение предсказанных запасов нефти во втором регионе в скважине составляет {predictions_valid_reg_1.mean()}')
print(f'RMSE модели составляет {result_reg_1}')

print('Третий регион')
print(f'Среднее значение предсказанных запасов нефти в первом регионе в скважине составляет {predictions_valid_reg_2.mean()}')
print(f'RMSE модели составляет {result_reg_2}')

Первый регион
Среднее значение предсказанных запасов нефти в первом регионе в скважине составляет 92.59256778438035
RMSE модели составляет 37.5794217150813
Второй регион
Среднее значение предсказанных запасов нефти во втором регионе в скважине составляет 68.728546895446
RMSE модели составляет 0.893099286775617
Третий регион
Среднее значение предсказанных запасов нефти в первом регионе в скважине составляет 94.96504596800489
RMSE модели составляет 40.02970873393434


**Вывод**

Были проанализированы следующие модели машинного обучения на всех датасетах: линейная регрессия, дерево решений, лес деревьев.

1) По первому датасету наилучшую сходимость показала модель линейной регрессии. Средняя квадратичная ошибка данной модели составила 37,57. 

2) По второму датасету наилучшую сходимость показала модель линейной регрессии. Средняя квадратичная ошибка данной модели составила 0,893. 

3) По третьему датасету наилучшую сходимость показала модель линейной регрессии. Средняя квадратичная ошибка данной модели составила 40,03. 

Также была подсчитано среднее количество запасов. По нему понятно, что модель 1 и 3 региона работает не очень хорошо. Ошибка составляет половину среднего значения. Тогда как во втором регионе модель работает хорошо.

## Подготовка к расчёту прибыли

In [14]:
points_drilling_total = 500 # точек для бурения всего
points_drilling_best = 200 # количество выбранных для бурения точек
total_money = 10**10 # бюджет
price_one_barrel = 450 # стоимость одного барреля
price_one_product = 450000 # стоимость продукции
interval = 0.025 # доверительный интервал
means = [predictions_valid_reg_0.mean(), predictions_valid_reg_1.mean(), predictions_valid_reg_2.mean()]

In [15]:
product_one_well_need = total_money / points_drilling_best / price_one_product
print('Среднее количество продукции для того, чтобы разработка окупилась', product_one_well_need)

count = 0

for mean in means:
    if product_one_well_need > mean:
        print(f'В {count} регионе разность между требуемым количеством нефти и среднием по скважинам составляет {product_one_well_need - mean}')
    count += 1

Среднее количество продукции для того, чтобы разработка окупилась 111.11111111111111
В 0 регионе разность между требуемым количеством нефти и среднием по скважинам составляет 18.518543326730764
В 1 регионе разность между требуемым количеством нефти и среднием по скважинам составляет 42.38256421566511
В 2 регионе разность между требуемым количеством нефти и среднием по скважинам составляет 16.146065143106227


**Вывод**

Согласно среднему предсказанному значению запасов нефти разработка никакого региона не окупится. Однако наилучшие перспективы у 3 региона, который отстает на 16 тыс. баррелей.

## Расчёт прибыли и рисков 

In [16]:
# функция для подсчета прибыли
def profit(probabilities, target, price=price_one_product, count=points_drilling_best):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    proruct_of_best_wells = selected.sum()
    profit = proruct_of_best_wells * price
    
    return profit

<div class="alert alert-success">
<h2> Комментарий ревьюера ✔️ <a class="tocSkip"> </h2>

Сортируем по прогнозам, берем факт 👍


In [17]:
probabilities_0 = pd.Series(predictions_valid_reg_0)
target_0 = pd.Series(target_valid_0)
target_0 = target_0.reset_index(drop=True)

probabilities_1 = pd.Series(predictions_valid_reg_1)
target_1 = pd.Series(target_valid_1)
target_1 = target_1.reset_index(drop=True)

probabilities_2 = pd.Series(predictions_valid_reg_2)
target_2 = pd.Series(target_valid_2)
target_2 = target_2.reset_index(drop=True)

**Расчет прибыли и рисков для первого региона**

In [18]:
state = np.random.RandomState(12345)   

values = []

for i in range(1000):
    target_0_subsample = target_0.sample(frac=points_drilling_total/len(target_0), replace=True, random_state=state)
    probabilities_0_subsample = probabilities_0[target_0_subsample.index]
   
    values.append(profit(probabilities_0_subsample, target_0_subsample))

values = pd.Series(values)

lower = values.quantile(0.025)
higher = values.quantile(0.975)

mean_profit = values.mean()

count = 0 
for value in values:
    if value < 10**10:
        count += 1

print(f'Средний доход первого региона равен {mean_profit / 10**9} млрд. рублей')
print(f'С вероятностью 95% доход равен от {higher / 10**9} до {lower / 10**9} млрд. рублей')
print(f'Средний прибыль первого региона равна {(mean_profit - 10**10) / 10**6} млн. рублей')
print(f'С вероятностью 95% прибыль равна от {(higher - 10**10) / 10**6} до {(lower - 10**10) / 10**6} млн. рублей')
print(f'Риск отрицательной прибыли в первом регионе {count / len(values) * 100} %')

Средний доход первого региона равен 10.425938526910594 млрд. рублей
С вероятностью 95% доход равен от 10.947976353358369 до 9.897909905162063 млрд. рублей
Средний прибыль первого региона равна 425.93852691059305 млн. рублей
С вероятностью 95% прибыль равна от 947.9763533583698 до -102.0900948379364 млн. рублей
Риск отрицательной прибыли в первом регионе 6.0 %


In [19]:
state = np.random.RandomState(12345)   

values = []

for i in range(1000):
    target_1_subsample = target_1.sample(frac=points_drilling_total/len(target_1), replace=True, random_state=state)
    probabilities_1_subsample = probabilities_1[target_1_subsample.index]
   
    values.append(profit(probabilities_1_subsample, target_1_subsample))

values = pd.Series(values)

lower = values.quantile(0.025)
higher = values.quantile(0.975)

mean_profit = values.mean()

count = 0 
for value in values:
    if value < 10**10:
        count += 1

print(f'Средний доход второго региона равен {mean_profit / 10**9} млрд. рублей')
print(f'С вероятностью 95% доход равен от {higher / 10**9} до {lower / 10**9} млрд. рублей')
print(f'Средний прибыль второго региона равна {(mean_profit - 10**10) / 10**6} млн. рублей')
print(f'С вероятностью 95% прибыль равна от {(higher - 10**10) / 10**6} до {(lower - 10**10) / 10**6} млн. рублей')
print(f'Риск отрицательной прибыли во втором регионе {count / len(values) * 100} %')

Средний доход второго региона равен 10.515222773443291 млрд. рублей
С вероятностью 95% доход равен от 10.931547591257049 до 10.068873225370503 млрд. рублей
Средний прибыль второго региона равна 515.2227734432907 млн. рублей
С вероятностью 95% прибыль равна от 931.5475912570496 до 68.87322537050247 млн. рублей
Риск отрицательной прибыли во втором регионе 1.0 %


In [20]:
state = np.random.RandomState(12345)   

values = []

for i in range(1000):
    target_2_subsample = target_2.sample(frac=points_drilling_total/len(target_2), replace=True, random_state=state)
    probabilities_2_subsample = probabilities_2[target_2_subsample.index]
   
    values.append(profit(probabilities_2_subsample, target_2_subsample))

values = pd.Series(values)

lower = values.quantile(0.025)
higher = values.quantile(0.975)

mean_profit = values.mean()

count = 0 
for value in values:
    if value < 10**10:
        count += 1

print(f'Средний доход третьего региона равен {mean_profit / 10**9} млрд. рублей')
print(f'С вероятностью 95% доход равен от {higher / 10**9} до {lower / 10**9} млрд. рублей')
print(f'Средний прибыль третьего региона равна {(mean_profit - 10**10) / 10**6} млн. рублей')
print(f'С вероятностью 95% прибыль равна от {(higher - 10**10) / 10**6} до {(lower - 10**10) / 10**6} млн. рублей')
print(f'Риск отрицательной прибыли в третьем регионе {count / len(values) * 100} %')

Средний доход третьего региона равен 10.435008362782757 млрд. рублей
С вероятностью 95% доход равен от 10.969706954180266 до 9.871119452670213 млрд. рублей
Средний прибыль третьего региона равна 435.0083627827568 млн. рублей
С вероятностью 95% прибыль равна от 969.7069541802654 до -128.88054732978821 млн. рублей
Риск отрицательной прибыли в третьем регионе 6.4 %


**Общий вывод**

С учетом рисков (риск отрицательной прибыли составляет 1%) для разработки выбирается **второй регион**. Также по данном региону прибыль является наибольшей и составляет 515 млн рублей. 

Также следует отметить, что в 95% доверительный интервал попадает только положительная прибыль.